In [1]:
#!pip install nerfacc
#!pip install nerfstudio

In [1]:
import torch
from torch import Tensor
import numpy as np
import nerfacc
from nerfstudio.model_components.ray_samplers import VolumetricSampler
from nerfacc.estimators.occ_grid import OccGridEstimator
# radiance_field = ...  # network: a NeRF model
# rays_o: Tensor = ...  # ray origins. (n_rays, 3)
# rays_d: Tensor = ...  # ray normalized directions. (n_rays, 3)

In [2]:
rays_o = torch.tensor([[1.1,1.1,0],[1.1,1.1,0.5]]).cuda()
rays_d = torch.tensor([[-2**(-0.5),-2**(-0.5),0],[-2**(-0.5),-2**(-0.5),0]]).cuda()

In [29]:
density_field = torch.zeros((32,32,32)).cuda()
density_field[:,:16,:16] = 0.03

In [30]:
def Part_1_By_2(x: torch.tensor):
    x &= 0x000003ff;                 # x = ---- ---- ---- ---- ---- --98 7654 3210
    x = (x ^ (x << 16)) & 0xff0000ff # x = ---- --98 ---- ---- ---- ---- 7654 3210
    x = (x ^ (x <<  8)) & 0x0300f00f # x = ---- --98 ---- ---- 7654 ---- ---- 3210
    x = (x ^ (x <<  4)) & 0x030c30c3 # x = ---- --98 ---- 76-- --54 ---- 32-- --10
    x = (x ^ (x <<  2)) & 0x09249249 # x = ---- 9--8 --7- -6-- 5--4 --3- -2-- 1--0
    return x

def morton_naive(x: torch.tensor, y: torch.tensor, z: torch.tensor):
    return Part_1_By_2(x) + (Part_1_By_2(y) << 1) + (Part_1_By_2(z) << 2)

def morton(input):
    return morton_naive(input[..., 0], input[..., 1], input[..., 2])

def inv_Part_1_By_2(x: torch.tensor):
    x = ((x >> 2) | x) & 0x030C30C3
    x = ((x >> 4) | x) & 0x0300F00F
    x = ((x >> 8) | x) & 0x030000FF
    x = ((x >>16) | x) & 0x000003FF
    return x

def inv_morton_naive(input: torch.tensor):
    x = input &        0x09249249
    y = (input >> 1) & 0x09249249
    z = (input >> 2) & 0x09249249
    
    return inv_Part_1_By_2(x), inv_Part_1_By_2(y), inv_Part_1_By_2(z)

def inv_morton(input:torch.tensor):
    x,y,z = inv_morton_naive(input)
    return torch.stack([x,y,z], dim = -1)

In [31]:
grid_raw = density_field

    #grid_raw = torch.ones_like(grid_raw, dtype = torch.float32, device = 'cuda')
grid = torch.zeros([32**3], dtype = torch.float32, device = 'cuda')
x, y, z = inv_morton_naive(torch.arange(0, 32**3, 1))
# for i,j,k in zip(x,y,z):
#     if i<16 and j < 16:
#         grid[i * 32 * 32 + j * 32 + k] = 0.5
grid[x * 32 * 32 + y * 32 + z] = grid_raw.permute(2,1,0).flatten()[x * 32 * 32 + y * 32 + z]
grid_3d = torch.reshape(grid > 0.01, [1, 32, 32, 32]).type(torch.bool)

estimator = OccGridEstimator(
    roi_aabb=[0, 0, 0, 1, 1, 1], resolution=32, levels=1
).cuda()

params_grid = {
    "resolution": torch.tensor([32, 32, 32], dtype = torch.int32),
    #"aabbs": torch.tensor([[-0.5, -0.5, -0.5, 1.5, 1.5, 1.5]]),
    "aabbs": torch.tensor([[0, 0, 0, 1, 1, 1]]),
    "occs":grid,
    "binaries": grid_3d
}
estimator.load_state_dict(params_grid)

<All keys matched successfully>

In [60]:
# def sigma_fn(
#    t_starts: Tensor, t_ends:Tensor, ray_indices: Tensor
# ) -> Tensor:
#     print('1')
#     """ Define how to query density for the estimator."""
#     t_origins = rays_o[ray_indices]  # (n_samples, 3)
#     t_dirs = rays_d[ray_indices]  # (n_samples, 3)
#     positions = t_origins + t_dirs * (t_starts + t_ends)[:, None] / 2.0
#     positions = positions.view(1,-1,1,1,3)
#     positions
#     sigmas = torch.nn.functional.grid_sample(density_field[None,None],positions,mode='bilinear',align_corners=True)
#     sigmas = sigmas.view(t_dirs.shape[0],-1)
#     return sigmas  # (n_samples,)

# Efficient Raymarching:
# ray_indices: (n_samples,). t_starts: (n_samples,). t_ends: (n_samples,).

ray_indices, t_starts, t_ends = estimator.sampling(
   rays_o, rays_d, near_plane=0.2, far_plane=2.0,render_step_size = 1e-1,
   early_stop_eps=0.01, alpha_thre=0.01,
)



In [61]:
x

tensor([ 0,  1,  0,  ..., 31, 30, 31])

In [62]:
y

tensor([ 0,  0,  1,  ..., 30, 31, 31])

In [63]:
ray_indices.shape

torch.Size([16])

In [64]:
ray_indices

tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [67]:
ray_indices.eq(0).sum()

tensor(8, device='cuda:0')

In [65]:
t_starts.shape

torch.Size([16])

In [59]:
rays_o[ray_indices] +  rays_d[ray_indices]* (t_starts[...,None] + t_ends[...,None])/2

tensor([[0.4990, 0.4990, 0.0000],
        [0.4282, 0.4282, 0.0000],
        [0.3575, 0.3575, 0.0000],
        [0.2868, 0.2868, 0.0000],
        [0.2161, 0.2161, 0.0000],
        [0.1454, 0.1454, 0.0000],
        [0.0747, 0.0747, 0.0000],
        [0.0040, 0.0040, 0.0000],
        [0.4990, 0.4990, 0.5000],
        [0.4282, 0.4282, 0.5000],
        [0.3575, 0.3575, 0.5000],
        [0.2868, 0.2868, 0.5000],
        [0.2161, 0.2161, 0.5000],
        [0.1454, 0.1454, 0.5000],
        [0.0747, 0.0747, 0.5000],
        [0.0040, 0.0040, 0.5000]], device='cuda:0')

In [28]:
(t_starts + t_ends).shape

torch.Size([0])

In [72]:
rays_o.shape

torch.Size([2, 3])

In [201]:
def near_far_from_aabb(rays_o, rays_d, aabb, min_near=0.05):
    # rays: [N, 3], [N, 3]
    # bound: int, radius for ball or half-edge-length for cube
    # return near [N, 1], far [N, 1]

    tmin = (aabb[:3] - rays_o) / (rays_d + 1e-15) # [N, 3]
    tmax = (aabb[3:]  - rays_o) / (rays_d + 1e-15)
    near = torch.where(tmin < tmax, tmin, tmax).amax(dim=-1, keepdim=True)
    far = torch.where(tmin > tmax, tmin, tmax).amin(dim=-1, keepdim=True)
    # if far < near, means no intersection, set both near and far to inf (1e9 here)
    mask = far < near
    near[mask] = 1e9
    far[mask] = 1e9
    # restrict near to a minimal value
    near = torch.clamp(near, min=min_near)

    return near, far

In [210]:
rays_o = torch.tensor([[0.7,0.7,0],[1.1,1.1,0.5]]).cuda()
rays_d = torch.tensor([[-2**(-0.5),-2**(-0.5),0],[-2**(-0.5),-2**(-0.5),0]]).cuda()
density_field = torch.zeros((32,32,32)).cuda()
density_field[:16,:16,:] = 1
aabb = torch.tensor([0.,0.,0.,1.,1.,1.]).cuda()

near,far= near_far_from_aabb(rays_o, rays_d, aabb)
mask  = (near != 1e9).squeeze().squeeze()
near = near[mask].squeeze().squeeze()
far = far[mask].squeeze()
rays_o = rays_o[mask]
rays_d = rays_d[mask]
step = (far-near)/32

In [211]:
grid_raw = density_field

grid = torch.zeros([32**3], dtype = torch.float32, device = 'cuda')
x, y, z = inv_morton_naive(torch.arange(0, 32**3, 1))
# for i,j,k in zip(x,y,z):
#     if i<16 and j < 16:
#         grid[i * 32 * 32 + j * 32 + k] = 0.5
grid[x * 32 * 32 + y * 32 + z] = grid_raw.flatten()[x * 32 * 32 + y * 32 + z]
grid_3d = torch.reshape(grid > 0.01, [1, 32, 32, 32]).type(torch.bool)

estimator = OccGridEstimator(
    roi_aabb=[0, 0, 0, 1, 1, 1], resolution=32, levels=1
).cuda()

params_grid = {
    "resolution": torch.tensor([32, 32, 32], dtype = torch.int32),
    #"aabbs": torch.tensor([[-0.5, -0.5, -0.5, 1.5, 1.5, 1.5]]),
    "aabbs": torch.tensor([[0, 0, 0, 1, 1, 1]]),
    "occs":grid,
    "binaries": grid_3d
}
estimator.load_state_dict(params_grid)

<All keys matched successfully>

In [219]:
ray_indices, t_starts, t_ends = estimator.sampling(
   rays_o, rays_d, near_plane=0.1, far_plane=3**(0.5),render_step_size = 1e-1,
   early_stop_eps=1, alpha_thre=0.01,
)

In [220]:
ray_indices

tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [221]:
t_starts

tensor([0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000, 0.9000, 0.8000, 0.9000,
        1.0000, 1.1000, 1.2000, 1.3000, 1.4000, 1.5000], device='cuda:0')